In [2]:
pip install numpy torch faiss-cpu transformers sentencepiece torchvision tqdm matplotlib


Looking in indexes: https://pypi.python.org/simple

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install ipywidgets


Looking in indexes: https://pypi.python.org/simple

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
import ipywidgets as widgets
widgets.IntSlider()

IntSlider(value=0)

In [5]:
pip install tqdm


Looking in indexes: https://pypi.python.org/simple

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [6]:
# Import necessary libraries
import os
import json
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import faiss
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from transformers import AutoTokenizer, AutoModelForCausalLM
from tqdm import tqdm
import matplotlib.pyplot as plt


In [7]:
# Define paths to the ARC dataset
arc_dataset_path = 'data/training'  # Adjust this path

# Function to load ARC tasks
def load_arc_tasks(task_dir):
    tasks = []
    for filename in os.listdir(task_dir):
        if filename.endswith('.json'):
            with open(os.path.join(task_dir, filename), 'r') as f:
                task = json.load(f)
                tasks.append(task)
    return tasks

# Load all tasks
all_tasks = load_arc_tasks(arc_dataset_path)
print(f"Total tasks loaded: {len(all_tasks)}")


Total tasks loaded: 400


In [8]:
# Shuffle the tasks
random.shuffle(all_tasks)

# Split into training and testing sets
split_ratio = 0.8
split_index = int(split_ratio * len(all_tasks))
train_tasks = all_tasks[:split_index]
test_tasks = all_tasks[split_index:]

print(f"Training tasks: {len(train_tasks)}")
print(f"Testing tasks: {len(test_tasks)}")


Training tasks: 320
Testing tasks: 80


In [9]:
# Define maximum grid size
MAX_GRID_SIZE = 30  # Adjust based on the dataset

# Function to preprocess grids
def preprocess_grid(grid, max_size=MAX_GRID_SIZE):
    height = len(grid)
    width = len(grid[0])
    # Initialize a grid filled with zeros
    processed_grid = np.zeros((max_size, max_size), dtype=np.int64)
    # Copy the original grid into the processed grid
    for i in range(height):
        for j in range(width):
            processed_grid[i, j] = grid[i][j]
    return processed_grid

# Build a set of all colors/symbols used in the dataset
symbols = set()
for task in train_tasks:
    for example in task['train']:
        for row in example['input']:
            symbols.update(row)
        for row in example['output']:
            symbols.update(row)

symbol_to_int = {symbol: idx for idx, symbol in enumerate(sorted(symbols))}
int_to_symbol = {idx: symbol for symbol, idx in symbol_to_int.items()}
num_symbols = len(symbol_to_int)
print(f"Total unique symbols: {num_symbols}")


Total unique symbols: 10


In [10]:
class GridEncoder(nn.Module):
    def __init__(self, embedding_dim=128, num_symbols=num_symbols):
        super(GridEncoder, self).__init__()
        self.embedding = nn.Embedding(num_symbols, 16)
        self.conv_layers = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),  # Reduces spatial dimensions by half
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),  # Reduces spatial dimensions by half again
        )
        # The fc layer will be initialized dynamically
        self.fc = None

    def forward(self, x):
        x = self.embedding(x)
        x = x.permute(0, 3, 1, 2)
        x = self.conv_layers(x)
        x = x.contiguous().view(x.size(0), -1)  # Use contiguous before view

        if self.fc is None:
            self.fc = nn.Linear(x.size(1), 128).to(x.device)

        x = self.fc(x)
        return x



In [11]:
class TripletGridDataset(Dataset):
    def __init__(self, tasks):
        self.samples = []
        for task in tasks:
            for example in task['train']:
                input_grid = preprocess_grid(example['input'])
                output_grid = preprocess_grid(example['output'])
                self.samples.append((input_grid, output_grid))

        # Create negatives by shuffling outputs
        self.negatives = [output for _, output in self.samples]
        random.shuffle(self.negatives)

        # Map symbols to integers
        self.symbol_to_int = symbol_to_int

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        anchor_grid, positive_grid = self.samples[idx]
        negative_grid = self.negatives[idx]

        # Convert grids to tensors and map symbols to integers
        anchor = torch.tensor(
            [[self.symbol_to_int.get(cell, 0) for cell in row] for row in anchor_grid],
            dtype=torch.long
        )
        positive = torch.tensor(
            [[self.symbol_to_int.get(cell, 0) for cell in row] for row in positive_grid],
            dtype=torch.long
        )
        negative = torch.tensor(
            [[self.symbol_to_int.get(cell, 0) for cell in row] for row in negative_grid],
            dtype=torch.long
        )

        return anchor, positive, negative


In [12]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cpu


In [13]:
# Instantiate the dataset and data loader
triplet_dataset = TripletGridDataset(train_tasks)
triplet_loader = DataLoader(triplet_dataset, batch_size=32, shuffle=True)

# Initialize the encoder and move it to the device
encoder = GridEncoder().to(device)

# Define the loss function and optimizer
criterion = nn.TripletMarginLoss(margin=1.0)
optimizer = optim.Adam(encoder.parameters(), lr=1e-3)

# Training loop
num_epochs = 5  # Adjust as needed

for epoch in range(num_epochs):
    encoder.train()
    total_loss = 0
    for batch_idx, (anchor, positive, negative) in enumerate(tqdm(triplet_loader, desc=f"Epoch {epoch+1}/{num_epochs}")):
        anchor = anchor.to(device)
        positive = positive.to(device)
        negative = negative.to(device)

        optimizer.zero_grad()
        anchor_emb = encoder(anchor)
        positive_emb = encoder(positive)
        negative_emb = encoder(negative)
        loss = criterion(anchor_emb, positive_emb, negative_emb)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(triplet_loader)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}")


Epoch 1/5: 100%|██████████| 33/33 [00:03<00:00,  8.76it/s]


Epoch 1/5, Loss: 0.6147


Epoch 2/5: 100%|██████████| 33/33 [00:02<00:00, 11.96it/s]


Epoch 2/5, Loss: 0.5050


Epoch 3/5: 100%|██████████| 33/33 [00:02<00:00, 12.11it/s]


Epoch 3/5, Loss: 0.4430


Epoch 4/5: 100%|██████████| 33/33 [00:02<00:00, 12.98it/s]


Epoch 4/5, Loss: 0.3903


Epoch 5/5: 100%|██████████| 33/33 [00:02<00:00, 12.95it/s]

Epoch 5/5, Loss: 0.3568


In [14]:
# Collect all input grids and their indices
all_input_grids = []
grid_indices = []
for task_idx, task in enumerate(train_tasks):
    for example_idx, example in enumerate(task['train']):
        input_grid = preprocess_grid(example['input'])
        all_input_grids.append(input_grid)
        grid_indices.append((task_idx, example_idx))

# Encode all input grids
encoder.eval()
embeddings = []
with torch.no_grad():
    for grid in tqdm(all_input_grids, desc="Encoding input grids"):
        grid_tensor = torch.tensor(
            [[symbol_to_int.get(cell, 0) for cell in row] for row in grid],
            dtype=torch.long
        ).unsqueeze(0).to(device)
        embedding = encoder(grid_tensor)
        embeddings.append(embedding.cpu().numpy())

# Convert embeddings to numpy array
embeddings_np = np.vstack(embeddings).astype('float32')

# Build the FAISS index
embedding_dim = embeddings_np.shape[1]
index = faiss.IndexFlatL2(embedding_dim)
index.add(embeddings_np)
print(f"FAISS index built with {index.ntotal} embeddings.")


Encoding input grids: 100%|██████████| 1051/1051 [00:00<00:00, 2406.81it/s]

FAISS index built with 1051 embeddings.


In [15]:
def retrieve_similar_examples(input_grid, k=4):
    encoder.eval()
    with torch.no_grad():
        grid_tensor = torch.tensor(
            [[symbol_to_int.get(cell, 0) for cell in row] for row in preprocess_grid(input_grid)],
            dtype=torch.long
        ).unsqueeze(0).to(device)
        input_embedding = encoder(grid_tensor).cpu().numpy()
    D, I = index.search(input_embedding, k)
    similar_examples = []
    for idx in I[0]:
        task_idx, example_idx = grid_indices[idx]
        sim_input_grid = all_input_grids[idx]
        sim_output_grid = preprocess_grid(train_tasks[task_idx]['train'][example_idx]['output'])
        similar_examples.append((sim_input_grid, sim_output_grid))
    return similar_examples


In [16]:
# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-125M")
model = AutoModelForCausalLM.from_pretrained("EleutherAI/gpt-neo-125M").to(device)


In [17]:
def grid_to_text(grid):
    grid = grid[:MAX_GRID_SIZE, :MAX_GRID_SIZE]
    text = '\n'.join([' '.join(map(str, row)) for row in grid])
    return text

def create_prompt(similar_examples, input_grid):
    prompt = "Below are examples of input grids and their corresponding output grids:\n\n"
    for idx, (sim_input, sim_output) in enumerate(similar_examples):
        prompt += f"Example {idx+1}:\nInput:\n{grid_to_text(sim_input)}\n"
        prompt += f"Output:\n{grid_to_text(sim_output)}\n\n"
    prompt += "Now, given the following input grid:\n"
    prompt += f"{grid_to_text(preprocess_grid(input_grid))}\n\n"
    prompt += "Please provide the corresponding output grid."
    return prompt


In [18]:
def generate_output(prompt):
    input_ids = tokenizer.encode(prompt, return_tensors='pt').to(device)
    max_length = min(1024, input_ids.shape[1] + 200)  # Adjust as needed
    output_ids = model.generate(
        input_ids,
        max_length=max_length,
        num_beams=5,
        no_repeat_ngram_size=2,
        early_stopping=True
    )
    output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return output_text[len(prompt):].strip()


In [19]:
def extract_output_grid(response_text):
    # Attempt to extract the grid after "Output:"
    if "Output:" in response_text:
        output_section = response_text.split("Output:")[-1].strip()
    else:
        output_section = response_text.strip()
    grid_lines = output_section.split('\n')
    output_grid = []
    for line in grid_lines:
        line = line.strip()
        if not line:
            continue
        try:
            row = [int(s) for s in line.strip().split()]
            output_grid.append(row)
        except ValueError:
            # Handle lines that don't contain integers
            continue
    if output_grid:
        return output_grid
    else:
        return None


In [ ]:
def update_embedding_database(input_grid, output_grid):
    encoder.eval()
    with torch.no_grad():
        input_tensor = torch.tensor(
            [[symbol_to_int.get(cell, 0) for cell in row] for row in preprocess_grid(input_grid)],
            dtype=torch.long
        ).unsqueeze(0).to(device)
        embedding = encoder(input_tensor).cpu().numpy()
    index.add(embedding)
    all_input_grids.append(preprocess_grid(input_grid))
    grid_indices.append((-1, -1))  # Indicate that this is a new addition
    # Optionally, you can store the output_grid if needed


: 

In [ ]:
# Minimal test code
try:
    # Test on a single example
    test_task = test_tasks[0]  # Select the first test task
    example = test_task['test'][0]  # Select the first example in the task

    print("Processing input grid...")
    input_grid = example['input']
    print(f"Input Grid: {input_grid}")

    expected_output = example.get('output', None)
    if expected_output:
        print(f"Expected Output: {expected_output}")
        
        print("Retrieving similar examples...")
        similar_examples = retrieve_similar_examples(input_grid)  # Ensure this is defined
        print(f"Similar Examples: {similar_examples}")

        print("Creating prompt...")
        prompt = create_prompt(similar_examples, input_grid)  # Ensure this is defined
        print(f"Prompt: {prompt}")

        print("Generating output...")
        output_text = generate_output(prompt)  # Ensure this is defined
        print(f"Output Text: {output_text}")

        print("Extracting output grid...")
        predicted_output = extract_output_grid(output_text)  # Ensure this is defined
        print(f"Predicted Output: {predicted_output}")

        is_correct = grids_are_equal(predicted_output, expected_output) if predicted_output else False
        print(f"Correct: {is_correct}")
    else:
        print("No expected output provided for this test task.")

except Exception as e:
    print(f"An error occurred: {e}")
